Imports

In [1]:
# Import the required libraries.
import cv2
import os
import glob
from PIL import Image
import random 
from random import randint
import numpy as Numpy
from numpy import asarray
from matplotlib import pyplot as plt
import splitfolders

In [11]:
# read folders
base_dir = "../Material-Recognition-Data/image/"
augmented_dir = "../Material-Recognition-Data/augmented_images/"
masked_dir = "../Material-Recognition-Data/mask/"
augmented_masked_dir = "../Material-Recognition-Data/augmented_masked_images/"

Augmenting Method

In [39]:
def fill(img, h, w):
    img = cv2.resize(img, (w, h), cv2.INTER_CUBIC)
    return img 

def zoom(img):
    val = random.uniform(0.5, 1)
    h, w = img.shape[:2]
    h_taken = int(val*h)
    w_taken = int(val*w)
    h_start = randint(0, h-h_taken)
    w_start = randint(0, w-w_taken)
    augmented_img = img[h_start:h_start+h_taken, w_start:w_start+w_taken, :]
    augmented_img = fill(augmented_img, h, w)
    return augmented_img

def vertical_shift(img):
    ratio = random.uniform(-0.7, 0.7)
    h, w = img.shape[:2]
    to_shift = h*ratio
    if ratio > 0: 
        augmented_img = img[:int(h-to_shift), :, :]
    else:
        augmented_img = img[int(-1*to_shift):, :, :]
    augmented_img = fill(augmented_img, h, w)
    return augmented_img

def horizontal_shift(img):
    ratio = random.uniform(-0.7, 0.7)
    h, w = img.shape[:2]
    to_shift = w*ratio
    if ratio > 0:
        augmented_img = img[:, :int(w-to_shift), :]
    else:
        augmented_img = img[:, int(-1*to_shift):, :]
    augmented_img = fill(augmented_img, h, w)
    return augmented_img

def crop_square(img):
    # target size is 128 for height, since we are going to crop out the extra width
    target_size = 128
    actual_size = img.shape[0]
    scale_value = target_size/actual_size

    # resize image
    resized_img = cv2.resize(img, (0, 0), fx=scale_value, fy=scale_value)
    
    # crop image to square
    center = resized_img.shape
    width = resized_img.shape[1]
    height = resized_img.shape[0]

    if width > height:
        desired_w = height 
        desired_h = height
    else:
        desired_w = width 
        desired_h = width

    x = height/2 - desired_h/2
    y = width/2 - desired_w/2 
    cropped_img = resized_img[int(x):int(x+desired_h), int(y):int(y+desired_w)]
    
    return cropped_img


def augment_images(images_list, mask_images_list, class_name, total_augment):
    class_dir = augmented_dir + class_name + "/"
    class_dir_masked = augmented_masked_dir + class_name + "/"
    images_per_class = total_augment 

    if not os.path.isdir(class_dir):
        os.makedirs(class_dir)

    if not os.path.isdir(class_dir_masked):
        os.makedirs(class_dir_masked)

    original_count = len(images_list)
    no_required_images = images_per_class - original_count
    print("Augmenting " + str(no_required_images) + " images for this category ...")

    # augment images
    while no_required_images > 0: 
        idx = no_required_images % original_count
        copy_img = images_list[idx]
        copy_mask_img = mask_images_list[idx]

        ran_choice = randint(1, 5)

        # apply diff transformations based on random selection
        if ran_choice == 1:
            # rotate counter clockwise
            augmented_img = cv2.flip(copy_img, 0)
            augmented_masked_img = cv2.flip(copy_mask_img, 0)
        elif ran_choice == 2:
            # rotate clockwise
            augmented_img = cv2.flip(copy_img, 1)
            augmented_masked_img = cv2.flip(copy_mask_img, 1)
        elif ran_choice == 3:
            # random zoom on image
           augmented_img = zoom(copy_img)
           augmented_masked_img = zoom(copy_mask_img)
        elif ran_choice == 4:
            # vertical shift image
            augmented_img = vertical_shift(copy_img)
            augmented_masked_img = vertical_shift(copy_mask_img)
        elif ran_choice == 5:
            # horizontal shift image
            augmented_img = horizontal_shift(copy_img)
            augmented_masked_img = horizontal_shift(copy_mask_img)
       
        # append to existing list
        images_list.append(augmented_img)
        mask_images_list.append(augmented_masked_img)
        # minus required images
        no_required_images -= 1

    # save all images to new dir 
    for idx in range(len(images_list)):
        curr_img = images_list[idx]

        curr_img = crop_square(curr_img)
        # convert BGR format to RGB format
        curr_img = cv2.cvtColor(curr_img, cv2.COLOR_BGR2RGB)

        # convert array back to img and save
        curr_img = Image.fromarray(curr_img)
        # pad index with 0s 
        img_idx = f'{idx+1:04}'
        img_name = class_name + "_" + img_idx + ".png"
        curr_img.save(class_dir + img_name)

    for idx in range(len(mask_images_list)):
        curr_img = mask_images_list[idx]
        curr_img = crop_square(curr_img)
        # convert BGR format to RGB format
        curr_img = cv2.cvtColor(curr_img, cv2.COLOR_BGR2RGB)

        # convert array back to img and save
        curr_img = Image.fromarray(curr_img)
        # pad index with 0s 
        img_idx = f'{idx+1:04}'
        img_name = class_name + "_" + img_idx + ".png"
        curr_img.save(class_dir_masked + img_name)


Read images and augment

In [40]:
image_folders = glob.glob(base_dir + "*")
image_folders = sorted(image_folders)

masked_folders = glob.glob(masked_dir + "*")
masked_folders = sorted(masked_folders)

for folder in range(len(image_folders)):
    # get class name
    class_name = os.path.basename(image_folders[folder])
    # get all paths of images in class
    class_path_list = glob.glob(base_dir + class_name + "/*")
    masked_class_path_list = glob.glob(masked_dir + class_name + "/*")

    # get list of images in class 
    images_list = [] 
    mask_images_list = []
    for idx in range(len(class_path_list)):
        if class_path_list[idx].endswith(".jpg"):
            curr_img = cv2.imread(class_path_list[idx])
            mask_img = cv2.imread(masked_class_path_list[idx])

            # convert image into numpy array
            if type(curr_img).__module__ != "numpy":
                curr_img = Numpy.asarray(curr_img)

            if type(mask_img).__module__ != "numpy":
                mask_img = Numpy.asarray(mask_img)

            
            
            images_list.append(curr_img)
            mask_images_list.append(mask_img)

    print("Current class that is being processed: " + class_name)
    print("Current amount of images in class: " + str(len(images_list)))
    # augment
    augment_images(images_list, mask_images_list, class_name, 1000)

    print("-------------------------------------------------------")

Current class that is being processed: fabric
Current amount of images in class: 100
Augmenting 900 images for this category ...
-------------------------------------------------------
Current class that is being processed: foliage
Current amount of images in class: 100
Augmenting 900 images for this category ...
-------------------------------------------------------
Current class that is being processed: glass
Current amount of images in class: 100
Augmenting 900 images for this category ...
-------------------------------------------------------
Current class that is being processed: leather
Current amount of images in class: 100
Augmenting 900 images for this category ...
-------------------------------------------------------
Current class that is being processed: metal
Current amount of images in class: 100
Augmenting 900 images for this category ...
-------------------------------------------------------
Current class that is being processed: paper
Current amount of images in cl

Save only 100 of each category for test set (original has 2.5k each category)

In [42]:
test_dir = "../Material-Recognition-Data/test_set/"
test_folders = glob.glob(test_dir + "*")
test_folders = sorted(test_folders)

final_test_dir = "../Material-Recognition-Data/final_test_set/"

for folder in range(len(test_folders)):
    class_name = os.path.basename(test_folders[folder])
    class_path_list = glob.glob(test_dir + class_name + "/*")

    class_dir = final_test_dir + class_name + "/"
    if not os.path.isdir(class_dir):
        os.makedirs(class_dir)

    # generate 100 random indexes
    idx_list = random.sample(range(len(class_path_list)), 100)

    for idx in idx_list:
        # read image
        curr_img = cv2.imread(class_path_list[idx])
        # convert image into numpy array
        if type(curr_img).__module__ != "numpy":
            curr_img = Numpy.asarray(curr_img)

        # target size is 128 for height, since we are going to crop out the extra width
        target_size = 128
        actual_size = curr_img.shape[0]
        scale_value = target_size/actual_size

        # resize image
        resized_img = cv2.resize(curr_img, (0, 0), fx=scale_value, fy=scale_value)

        # convert BGR format to RGB format
        curr_img = cv2.cvtColor(resized_img, cv2.COLOR_BGR2RGB)

        # convert array back to img and save
        curr_img = Image.fromarray(curr_img)
        # pad index with 0s 
        img_idx = f'{idx+1:06}'
        img_name = class_name + "_" + img_idx + ".png"
        curr_img.save(class_dir + img_name)

